In [1]:
import xlwings as xw
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Connect to the open workbook "ES RTD.xlsx"
wb = xw.Book('ES RTD.xlsx')  # Adjust to the correct file path

# Extract ES Spot Price from wb.sheets['Watchlist'] cell C4
ws_watchlist = wb.sheets['Watchlist']
ES_spot = ws_watchlist.range('C4').value

# Access the specific sheet "ES RTD"
ws_es = wb.sheets['ES RTD']

# Extract dynamic date and time from cell A1
date_info = ws_es.range('A1').value
date_part = date_info.split(' on ')[1].split(' ')[0]

# Extract ES Ticker between "for" and "on" in cell A1 (e.g., '/ESZ24')
ticker = date_info.split('for ')[1].split(' on')[0]

# Define the start row (headers start in row 6, data in row 7)
start_row = 7

# Automatically detect the last row with useful data (assuming column A reliably contains data)
row = start_row
while True:
    value = ws_es.range(f'C{row}').value
    if not value:  # Stop when an empty cell is found
        end_row = row - 1  # Set the end row to the last non-empty row
        break
    row += 1

print(f"Start row: {start_row}, End row: {end_row}")

# Initialize lists to store strikes and gamma exposures for Calls and Puts
strikes = []
call_gamma_exposure_list = []
put_gamma_exposure_list = []
call_volumes = []
put_volumes = []

# Loop through each row and extract relevant data for Calls and Puts
for row in range(start_row, end_row + 1):
    strike_price = ws_es.range(f'Q{row}').value
    call_gamma = ws_es.range(f'I{row}').value
    call_open_int = ws_es.range(f'K{row}').value
    put_gamma = ws_es.range(f'AB{row}').value
    put_open_int = ws_es.range(f'AD{row}').value
    call_volume = ws_es.range(f'G{row}').value  # Volume for Calls
    put_volume = ws_es.range(f'Z{row}').value  # Volume for Puts
    
    if call_gamma and call_open_int:
        call_gamma_exposure = call_gamma * call_open_int * (ES_spot)**2 * 0.01
    else:
        call_gamma_exposure = 0
    
    if put_gamma and put_open_int:
        put_gamma_exposure = put_gamma * put_open_int * (ES_spot)**2 * 0.01 * -1
    else:
        put_gamma_exposure = 0
    
    strikes.append(strike_price)
    call_gamma_exposure_list.append(call_gamma_exposure)
    put_gamma_exposure_list.append(put_gamma_exposure)
    call_volumes.append(call_volume)
    put_volumes.append(put_volume)

# Define the ±350 range from the ES Spot Price
minStrike = ES_spot - 350
maxStrike = ES_spot + 350

# Create a DataFrame
df = pd.DataFrame({
    'Strike': strikes,
    'Call Gamma Exposure': call_gamma_exposure_list,
    'Put Gamma Exposure': put_gamma_exposure_list,
    'Call Volume': call_volumes,
    'Put Volume': put_volumes
})

# Filter the DataFrame to only show strikes within the ±350 range of ES Spot Price
df_filtered = df[(df['Strike'] >= minStrike) & (df['Strike'] <= maxStrike)].copy()  # Explicitly create a copy

# Use .loc[] to avoid SettingWithCopyWarning
df_filtered.loc[:, 'Total Gamma Exposure'] = df_filtered['Call Gamma Exposure'] + df_filtered['Put Gamma Exposure']

# ---- Find Gamma Flip Point ----
totalGamma_filtered = df_filtered['Total Gamma Exposure'].values
levels_filtered = df_filtered['Strike'].values
zeroCrossIdx = np.where(np.diff(np.sign(totalGamma_filtered)))[0]

# Check if there's any zero-crossing index
if zeroCrossIdx.size > 0:
    # Store potential flip points
    flip_points = []
    for idx in zeroCrossIdx:
        negGamma = totalGamma_filtered[idx]
        posGamma = totalGamma_filtered[idx + 1]
        negStrike = levels_filtered[idx]
        posStrike = levels_filtered[idx + 1]

        # Linear interpolation to find the exact Gamma Flip point
        interpolated_flip = posStrike - ((posStrike - negStrike) * posGamma / (posGamma - negGamma))
        flip_points.append(interpolated_flip)

    # If multiple flip points, find the one closest to the ES Spot Price
    flip_points = np.array(flip_points)
    closest_flip_idx = np.argmin(np.abs(flip_points - ES_spot))
    zeroGamma = flip_points[closest_flip_idx]
else:
    zeroGamma = None
    print("Warning: No Gamma Flip Point found. Skipping the Gamma Flip line in the chart.")

# ---- Chart Creation Section ----

# Initialize the figure
fig = go.Figure()

# Create the bar chart (Gamma Exposure)
bar_chart_call = go.Bar(x=df_filtered['Strike'], y=df_filtered['Call Gamma Exposure'], name='Call Gamma', marker_color='blue')
bar_chart_put = go.Bar(x=df_filtered['Strike'], y=df_filtered['Put Gamma Exposure'], name='Put Gamma', marker_color='red')
bar_chart_total = go.Bar(x=df_filtered['Strike'], y=df_filtered['Total Gamma Exposure'], name='Total Gamma', marker_color='green')

# Create the volume distribution chart (Volume by Strike Price)
volume_call = go.Bar(x=df_filtered['Strike'], y=df_filtered['Call Volume'], name='Call Volume', marker_color='blue')
volume_put = go.Bar(x=df_filtered['Strike'], y=df_filtered['Put Volume'], name='Put Volume', marker_color='red')

# Add a vertical line for ES Spot Price
y_min = min(df_filtered['Call Gamma Exposure'].min(), df_filtered['Put Gamma Exposure'].min())
y_max = max(df_filtered['Call Gamma Exposure'].max(), df_filtered['Put Gamma Exposure'].max())
spot_price_line = go.Scatter(
    x=[ES_spot, ES_spot],  # Constant x (ES Spot Price)
    y=[y_min, y_max],  # Updated y-range
    mode='lines',
    name=f'ES Spot Price: {ES_spot}',
    line=dict(color='black', width=3, dash='solid')  # Style the line
)

# ---- Add Gamma Flip line to all charts ----
if zeroGamma:
    gamma_flip_line = go.Scatter(
        x=[zeroGamma, zeroGamma],  # Constant x (Gamma Flip Strike)
        y=[y_min, y_max],  # Updated y-range
        mode='lines',
        name=f'Gamma Flip: {zeroGamma}',
        line=dict(color='purple', width=3, dash='dash')  # Style the line
    )

# Add traces to the figure
fig.add_traces([bar_chart_call, bar_chart_put, volume_call, volume_put, bar_chart_total, spot_price_line])

# Add Gamma Flip line to all charts
if zeroGamma:
    fig.add_trace(gamma_flip_line)

# Define the dropdown menu options
dropdown_buttons = [
    dict(label='Gamma Exposure (Bar)',
         method='update',
         args=[{'visible': [True, True, False, False, False, True, zeroGamma is not None]},
               {'title': f'Gamma Exposure by Strike Price (ES) - {date_part}'}]),
    dict(label='Volume Distribution',
         method='update',
         args=[{'visible': [False, False, True, True, False, False, False]}, 
               {'title': f'Volume Distribution by Strike Price (ES) - {date_part}'}]),
    dict(label='Total Gamma Exposure',
         method='update',
         args=[{'visible': [False, False, False, False, True, True, zeroGamma is not None]},
               {'title': f'Total Gamma Exposure by Strike Price (ES) - {date_part}'}])
]

# Initialize all traces to be invisible except the default (Gamma Exposure bar chart)
fig.update_traces(visible=False)
fig.data[0].visible = True  # Call Gamma (Bar)
fig.data[1].visible = True  # Put Gamma (Bar)
fig.data[5].visible = True  # Spot Price Line
if zeroGamma:
    fig.data[-1].visible = True  # Gamma Flip line


# Add dropdown menu
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=dropdown_buttons,
        x=1.15,  # Positioning the dropdown
        y=1.15,
        xanchor='right'
    )]
)

# Update layout
fig.update_layout(
    title=f'Gamma Exposure by Strike Price (ES) - {date_part}',
    xaxis_title='Strike Price',
    yaxis_title='Gamma Exposure',
    barmode='group',
    xaxis_tickangle=-45
)

# Show the figure with dropdown
fig.show()

###Saving function for NT indicator use

# Assuming df_filtered and zeroGamma already exist and have the necessary data
# Sorting the DataFrame for highest and lowest total gamma exposures
top_5 = df_filtered.nlargest(5, 'Total Gamma Exposure').copy()  # Top 5 highest total gamma exposure
bottom_5 = df_filtered.nsmallest(5, 'Total Gamma Exposure').copy()  # Top 5 lowest total gamma exposure

# Assigning labels to the top 5 using .loc[] and ensuring it's not a view modification
top_5.loc[:, 'Gamma Level'] = [f'Call Gamma {i+1}' for i in range(5)]

# Assigning labels to the bottom 5 using .loc[]
bottom_5.loc[:, 'Gamma Level'] = [f'Put Gamma {i+1}' for i in range(5)]

# Concatenate the top and bottom gamma exposures, making sure both DataFrames are copies
top_bottom_combined = pd.concat([top_5.copy(), bottom_5.copy()])

# Selecting the required columns: Strike, Total Gamma Exposure, Gamma Level
output_df = top_bottom_combined[['Strike', 'Total Gamma Exposure', 'Gamma Level']].copy()

# Convert Total Gamma Exposure to millions, rounded to 2 decimal places using .loc[] to avoid SettingWithCopyWarning
output_df.loc[:, 'Total Gamma Exposure'] = (output_df['Total Gamma Exposure'] / 1_000_000).round(2)

# Add a new column for "Gamma Flip" and fill it with the gamma flip value (zeroGamma)
output_df.loc[:, 'Gamma Flip'] = float(zeroGamma)  # Ensure it's explicitly cast to float

# Saving to CSV file, applying formatting while saving to maintain float64 dtype during processing
output_df.to_csv(r'Your_path_here\NinjaTrader 8\bin\Custom\ESgammaLVL.csv', index=False, float_format='%.2f')
print("Top and bottom gamma levels with Gamma Flip saved to 'ESgammaLVL.csv'")


Start row: 7, End row: 387


Top and bottom gamma levels with Gamma Flip saved to 'ESgammaLVL.csv'
